In [1]:
import random
import pandas as pd
import numpy as np
import os

from sklearn.ensemble import RandomForestRegressor

import warnings
warnings.filterwarnings(action='ignore') 

In [2]:
def seed_everything(seed):
    '''
    fix random seed
    '''
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)

seed_everything(42) # Seed 고정

In [19]:
# Feature engineering 1 - scaling --> effects none ...

In [7]:
'''
feature extraction
'''

train_df = pd.read_csv('./data/train.csv')
test_df = pd.read_csv('./data/test.csv')

train_df = train_df.fillna(0)


#시계열 특성을 학습에 반영하기 위해 일시를 월, 일, 시간으로 나눕니다
train_df['month'] = train_df['일시'].apply(lambda x : int(x[4:6]))
train_df['day'] = train_df['일시'].apply(lambda x : int(x[6:8]))
train_df['time'] = train_df['일시'].apply(lambda x : int(x[9:11]))


train_x = train_df.drop(columns=['num_date_time', '일시', '일조(hr)', '일사(MJ/m2)', '전력소비량(kWh)'])

train_y = train_df['전력소비량(kWh)']

# Model Fit

In [36]:
'''
random forest fit
'''
# model = RandomForestRegressor()
# model.fit(reduced_train_x, train_y)

RandomForestRegressor()

In [8]:
# K-Fold
from sklearn.model_selection import KFold

n_splits = 5  # 폴드 개수
kf = KFold(n_splits=n_splits, shuffle=True, random_state=0)

best_model = None
best_score = -float('inf')  # 초기화: 음수 무한대로 설정

for train_idx, test_idx in kf.split(train_df):
    X_train, X_test = train_x.to_numpy()[train_idx], train_x.to_numpy()[test_idx]
    y_train, y_test = train_y.to_numpy()[train_idx], train_y.to_numpy()[test_idx]

    # 모델 훈련 및 평가 (여기에서는 간단한 선형 회귀 모델 사용)
    model = RandomForestRegressor()
    model.fit(X_train, y_train)
    score = model.score(X_test, y_test)

    if score > best_score:
        best_score = score
        best_model = model

    print("Test Score:", score)

Test Score: 0.983083964696658
Test Score: 0.9822495789233363
Test Score: 0.9829883474920242
Test Score: 0.9822677342016192


In [58]:
train_x

,건물번호,기온(C),풍속(m/s),습도(%),일조(hr),일사(MJ/m2),month,day,time
0,1,18.6,0.9,42.0,0.0,0.0,6,1,0
1,1,18.0,1.1,45.0,0.0,0.0,6,1,1
2,1,17.7,1.5,45.0,0.0,0.0,6,1,2
3,1,16.7,1.4,48.0,0.0,0.0,6,1,3
4,1,18.4,2.8,43.0,0.0,0.0,6,1,4
...,...,...,...,...,...,...,...,...,...
203995,100,23.1,0.9,86.0,0.5,0.0,8,24,19
203996,100,22.4,1.3,86.0,0.0,0.0,8,24,20
203997,100,21.3,1.0,92.0,0.0,0.0,8,24,21
203998,100,21.0,0.3,94.0,0.0,0.0,8,24,22


# Inference & Submission

In [59]:
### Test set pre-process
test_df = pd.read_csv('./data/test.csv')

test_df['month'] = test_df['일시'].apply(lambda x : int(x[4:6]))
test_df['day'] = test_df['일시'].apply(lambda x : int(x[6:8]))
test_df['time'] = test_df['일시'].apply(lambda x : int(x[9:11]))

test_x = test_df.drop(columns=['num_date_time', '강수량(mm)','일시'])


In [63]:
test_df

,num_date_time,건물번호,일시,기온(C),강수량(mm),풍속(m/s),습도(%),month,day,time
0,1_20220825 00,1,20220825 00,23.5,0.0,2.2,72,8,25,0
1,1_20220825 01,1,20220825 01,23.0,0.0,0.9,72,8,25,1
2,1_20220825 02,1,20220825 02,22.7,0.0,1.5,75,8,25,2
3,1_20220825 03,1,20220825 03,22.1,0.0,1.3,78,8,25,3
4,1_20220825 04,1,20220825 04,21.8,0.0,1.0,77,8,25,4
...,...,...,...,...,...,...,...,...,...,...
16795,100_20220831 19,100,20220831 19,22.5,0.0,0.9,84,8,31,19
16796,100_20220831 20,100,20220831 20,20.7,0.0,0.4,95,8,31,20
16797,100_20220831 21,100,20220831 21,20.2,0.0,0.4,98,8,31,21
16798,100_20220831 22,100,20220831 22,20.1,0.0,1.1,97,8,31,22


In [60]:
# inference
preds = model.predict(test_x)


ValueError: X has 7 features, but RandomForestRegressor is expecting 9 features as input.

In [40]:
# sub mission
submission = pd.read_csv('./data/sample_submission.csv')
submission['answer'] = preds
submission.head(5)

,num_date_time,answer
0,1_20220825 00,1114.4688
1,1_20220825 01,1076.0040
2,1_20220825 02,1120.3296
3,1_20220825 03,634.8888
4,1_20220825 04,938.9040


In [41]:
submission.to_csv("./data/submission_random_forest.csv", index=False)